<div align='left'><font size='6' color='#5b2c6f '> Purpose of Notebook</font></div>


### Building a NLP model to predict which brand a new product should be assigned.

- [Functions](#1)
- [Import Packages](#2)
- [Explore Data](#3)

## [Functions](#1)<a id='1'></a>

## [Import Packages](#2)<a id='2'></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




## [Explore data](#3)<a id='3'></a>

In [4]:
# read in data
behold_brand = pd.read_csv('data/behold_brands USC.csv')
behold_product = pd.read_excel('data/Behold+product+data+04262021.xlsx')
outfit_combo = pd.read_csv('data/outfit_combinations USC.csv')
tags = pd.read_csv('data/usc_additional_tags USC.csv')
print(f'''
Brand shape: {behold_brand.shape}.
Product shape: {behold_product.shape}.
Outfit shape: {outfit_combo.shape}.
Tags shaape: {tags.shape}.
''')


Brand shape: (162, 10).
Product shape: (61355, 11).
Outfit shape: (5291, 5).
Tags shaape: (97420, 4).

